In [ ]:
mpg<-read.table("auto-mpg.txt",col.names = c("mpg","cylinders","displacement","horsepower","weight","acceleration",
                                             "year","origin","name"),colClasses = c("numeric","integer","numeric",
                                                                                    "numeric","numeric","numeric",
                                                                                    "integer","factor","character"),
                                                                                     na.strings = "?")
str(mpg)
summary(mpg)
mpg[c(1,21,32),]    #read data

There are 6 obs with `horsepower` missing. Since this is a small number comparing to $N=398$, we decide to omit the missing data.

In [ ]:
mpg<-na.omit(mpg)        #delete missing data

In [ ]:
car::scatterplotMatrix(mpg[,-9])     #scatter plot matrix of data

We first notice that the relationship between the response `mpg` and many predictors are somewhat curved (nonlinear). This suggests transformation of variables may be necessary.

In [ ]:
corrplot::corrplot(cor(mpg[,-c(8,9)]))   #correlation of data 

There are also strong correlations among the predictors, especially `cylinders`, `displacement`, `horsepower`, and `weight`. We further check the **Variance Inflation Factor**.

In [ ]:
car::vif(lm(mpg~cylinders+displacement+horsepower+weight+acceleration+year,data=mpg))   #check multilinearity

Many of the predictors have a VIF greater than 10. For example, `displacement` can be predicted by other variables to a large extent. It is therefore necessary to do variable selection.

# Variable Transformation

We first choose the transformation of predictors.This is meant to transform the predictors into **Multinormality**.

In [ ]:
summary(car::powerTransform(mpg[,3:6]))      #estimate transformation parameter for multinormal

Since none of the predictors range over many orders of magnitude, we chose not to transform the predictors to maintain interpretability. However, from the scatter plot matrix, we see that there seems to be a nonlinear relationship between the response `mpg` and many of the predictors. We then use the Box-Cox transformation and try to find the $\lambda$ that minimizes the $RSS(\lambda)$ (Residual Sum of Squares). This is realized by the `MASS::boxcox()` function.

In [ ]:
car::inverseResponsePlot(lm(mpg~.,data=mpg[,-9]))

In [ ]:
MASS::boxcox(lm(mpg~.,data=mpg[,-9]))      #box-cox transformation for response

The estimated $\hat \lambda$ is -0.37. It is plausible to consider $\lambda=0$, $\lambda=-0.5$, or $\lambda=-1$. Nonetheless, we preferred to use the transformation given by $\lambda=-1$, namely $1/Y$, because it gives a very natural interpretation. Other than the US who uses $\textbf {mpg}$, most areas use the unit $\textbf {L/100km}$ for fuel consumption, which is just proportional to the inverse of its value in mpg.

In [ ]:
mpg$lp100km<-100/mpg$mpg*0.621371192*3.78541178
#car::inverseResponsePlot(lm(lp100km~., data=mpg[,-c(1,9)]))  

In [ ]:
car::scatterplotMatrix(mpg[,-9])
corrplot::corrplot(cor(mpg[,-c(8,9)]))

We hereafter choose `lp100km` as the response variable instead of `mpg`.

# Variable Selection via Best Subset Regression

In [ ]:
lm.subset<-leaps::regsubsets(lp100km~.,data=mpg[,-c(1,9)])
#lm.subset<-leaps::regsubsets(lp100km~.,data=mpg[,c(2:5,10)])
print(summary.subset<-summary(lm.subset))    #all subset regression 

In [ ]:
X<-as.matrix(cbind(mpg[,2:7],as.numeric(mpg$origin==2),as.numeric(mpg$origin==3)))
subset.list<-list(4,c(4,6),c(3,4,6),c(3,4,6,7),c(3,4,5,6,7),c(1,3,4,5,6,7),c(1,2,3,4,6,7,8),1:8)

5-fold CV

In [ ]:
CV.kFolds<-function(subset,k){
    Y<-mpg$lp100km
    set.seed(5)
    cvSplits<-caret::createFolds(mpg$lp100km,k=k)
    e<-rep(0,nrow(mpg))
    for (i in 1:k){
        beta.i<-as.vector(lm(Y[-cvSplits[[i]]]~X[-cvSplits[[i]],subset])$coef)
        #print(X[-cvSplits[[i]],subset])
        #print(beta.i)
        e[cvSplits[[i]]]<-Y[cvSplits[[i]]]-as.vector(cbind(rep(1,length(cvSplits[[i]])),X[cvSplits[[i]],subset])%*%beta.i)
        #print(e)
    }
    
    mean(e^2)
}

CV.kFolds<-Vectorize(CV.kFolds)
CV.5folds<-CV.kFolds(subset.list,5)     #5-fold CV criterion for model selection

In [ ]:
#data.frame(adjr2=summary.subset$adjr2,cp=summary.subset$cp, bic=summary.subset$bic)
display=cbind(as.data.frame(summary.subset$outmat),round(summary.subset$rsq,4),round(summary.subset$adjr2,4),
              round(summary.subset$cp,4), round(summary.subset$bic,4),round(CV.5folds,4))
colnames(display)[9:13]=c("R^2","adj R^2","Cp","BIC","5-fold CV")
display         #model selection result for different methods

Therefore, the chosen model (based on BIC and 5-fold CV) is the one with three predictors: `horsepower`, `weight`, and `year`.

In [ ]:
summary(lm.chosen<-lm(lp100km~horsepower+weight+year,data=mpg))    #Chosen Model

# Regression Diagnostic Based on the Chosen Model

In [ ]:
plot(lm.chosen,which=1:6)     #Regression Diagnostics

Have a look at those observations with large studentized residuals or large cook distance, which are possible outliers.

In [ ]:
which(abs(rstudent(lm.chosen))>3)
mpg[c(which(abs(rstudent(lm.chosen))>3),116),] 
lm.chosen$fitted.values[c(which(abs(rstudent(lm.chosen))>3),116)]    #outliers

In [ ]:
plot(mpg$weight, rstandard(lm.chosen))
abline(0,0)                  #standardized residuals plot

In [ ]:
car::residualPlots(lm.chosen)  

In [ ]:
summary(lm.chosen.refit<-lm(lp100km~horsepower+weight+year,data=mpg[-c(which(abs(rstudent(lm.chosen))>3),116),]))
plot(lm.chosen.refit,which=1:6)    #summary of chosen model

# Analysis of Variance and Analysis of Covariance: Does `Origin` matter?

In [ ]:
summary(lm.origin.anova<-lm(lp100km~origin,data=mpg))
anova(lm.origin.anova)   #anova

It seems that the means of the fuel consumption do differ from different origins. However, after we consider other predictors in the model, origin no longer matters in terms of prediction.

In [ ]:
summary(lm.origin.ancova<-lm(lp100km~horsepower+year+weight+origin,data=mpg))
anova(lm.origin.ancova)#ancova

When given the other predictors, origin3 will now be insignificant. However, origin2 remains significant, which indicates that `origin=Europe` itself contributes to a lower fuel consumption. A possible explanation is that European cars focus more on fuel economy (becuase of higher fuel price in Europe), and engines made in Europe, especially German, deployed more advance technology that would save fuel.

# Principle Component Analysis

In [ ]:
PCA<-prcomp(mpg[,2:7],scale. = TRUE)
#PCA<-princomp(mpg[,2:7],cor = TRUE)
summary(PCA)
#PCA$loadings
round(PCA$rotation,3)  #the coefficients of principle components on the origin predictors

In [ ]:
PCA.z<-PCA$x
PCA.z      #convert data into the score of principle components

In [ ]:
lm.fit.pcr2<-lm(mpg$lp100km~1+PCA.z[,1]+PCA.z[,2])           #use first two components to do regression
summary(lm.fit.pcr2)
lm.fit.pcr3<-(lm(mpg$lp100km~1+PCA.z[,1]+PCA.z[,2]+PCA.z[,3]))
summary(lm.fit.pcr3)                                         #use first three components to do regression